In [1]:
from __future__ import print_function

import os
import re
import cPickle, gzip
import progressbar
from rdkit.Chem import AllChem
import parser.Smipar as Smipar

In [2]:
# rsmi data reloader

data_directory = 'data'
processed_data_filename = 'processed_data.pkl.gz'
processed_data_filepath = os.path.join(data_directory, processed_data_filename)
data_length = 1094235

bar = progressbar.ProgressBar(max_value=data_length)

rsmi_data = []

with gzip.open(processed_data_filepath, 'rb') as data_file:
    i = 1
    while 1:
        try:
            line, ref = cPickle.load(data_file)
            rsmi_data.append((line, ref))
        except EOFError:
            break
        bar.update(i)
        i += 1

 99% (1088854 of 1094235) |################ | Elapsed Time: 0:00:13 ETA: 0:00:00

In [3]:
# data reloader

with gzip.open('data/vocab/vocab_dict.pkl.gz', 'rb') as dict_file:
    vocab_reactants, vocab_products = cPickle.load(dict_file)

with gzip.open('data/vocab/vocab_list.pkl.gz', 'rb') as list_file:
    reactants_token_list, products_token_list = cPickle.load(list_file)

In [7]:
bar = progressbar.ProgressBar(max_value=data_length)

token_len = []

with gzip.open('data/training_data.pkl.gz', 'wb') as training_file:
    
    for i, rsmi in enumerate(rsmi_data):

        rsmi = rsmi[0]

        reactant_list = []
        agent_list = []
        product_list = []

        split_rsmi = rsmi.split('>')
        reactants = split_rsmi[0].split('.')
        agents = split_rsmi[1].split('.')
        products = split_rsmi[2].split('.')

        for reactant in reactants:
            reactant_list += Smipar.parser_list(reactant)
            reactant_list += '.'
        for agent in agents:
            agent_list += Smipar.parser_list(agent)
            agent_list += '.'
        for product in products:
            product_list += Smipar.parser_list(product)
            product_list += '.'

        reactant_list.pop() # to pop last '.'
        agent_list.pop()
        product_list.pop()

        reactant_list += '>'
        reactant_list += agent_list
        
        token_len.append((len(reactant_list), len(product_list)))

        cPickle.dump(([reactants_token_list.index(r) for r in reactant_list],
                      [products_token_list.index(p) for p in product_list],
                      rsmi[1]), training_file, 2)
                     
        bar.update(i)

 99% (1094232 of 1094235) |################ | Elapsed Time: 3:42:20 ETA: 0:00:00

In [8]:
trans = zip(*token_len)
print(max(trans[0]), max(trans[1]))

149 79


In [12]:
print("total:", len(token_len))
count = [0, 0, 0, 0]
for length in token_len:
    if(length[0] < 150 and length[1] < 80):
        count[3] += 1
        if(length[0] < 90 and length[1] < 65):
            count[2] += 1
            if(length[0] < 70 and length[1] < 60):
                count[1] += 1
                if(length[0] < 54 and length[1] < 54):
                    count[0] += 1
print("cut:", count)

total: 1094234
1094233
cut: [378038, 662769, 896069, 1094234]


In [3]:
count=0
bar = progressbar.ProgressBar(max_value=1094234)
with gzip.open('data/training_data.pkl.gz', 'rb') as data_file:
    with gzip.open('data/train.pkl.gz', 'wb') as train_file:
        with gzip.open('data/dev.pkl.gz', 'wb') as dev_file:
            i = 1
            while 1:
                try:
                    reactants, products, _ = cPickle.load(data_file)
                    if i%100:
                        cPickle.dump((reactants, products), train_file, 2)
                    else:
                        cPickle.dump((reactants, products), dev_file, 2)
                        count += 1
                except EOFError:
                    break
                bar.update(i)
                i += 1

import time
time.sleep(1)

print(count)

 99% (1092670 of 1094234) |################ | Elapsed Time: 0:03:23 ETA: 0:00:00

10942


100% (1094234 of 1094234) |################| Elapsed Time: 0:03:24 ETA:  0:00:00